In [1]:
import csv
import numpy as np
from pprint import pprint
from sturdr.nav.ephemeris import GetNavStates
from sturdr.nav.estimation import LeastSquares, LeastSquaresPos
from sturdr.utils.constants import LIGHT_SPEED, GPS_L1CA_CARRIER_FREQ, GPS_L1CA_CODE_FREQ
from sturdr.utils.coordinates import ecef2lla, eci2ecef
from sturdr.dsp.discriminator import DllVariance

BETA = LIGHT_SPEED / GPS_L1CA_CODE_FREQ
LAMBDA = LIGHT_SPEED / GPS_L1CA_CARRIER_FREQ

# PRN = [1, 7, 14, 17, 19, 21, 30]
ToW = 413658.0
CNo = 10**(np.asarray([36.826, 41.229, 43.359, 36.599, 36.312, 43.221, 42.617])/10)
code_phase = np.array([11421.60386881124, 8209.110011406465, 11324.089926379798, 10902.806345747738, 3954.6763081617355, 4235.6660033346, 12222.95770424594])
code_phase_time = code_phase / GPS_L1CA_CODE_FREQ
samples = np.asarray([222618, 160675, 221518, 213405, 76880, 82941, 238359])
# samples += np.asarray([66, 960, 117, 122, 0, 855, 687])
# transmit_time = ToW + (samples / 20e6)
transmit_time = ToW + code_phase_time
receive_time = transmit_time.max() + 0.068802 # common receive time/sample including transmit time
tgd = np.zeros(7)
ephem = {}

with open("../results/GPS_L1CA_TEST_Ephemeris.csv", "r") as file:
    reader = csv.DictReader(file)
    for i,item in enumerate(reader):
        item.pop('id', None)
        item.pop('iode', None)
        item.pop('iodc', None)
        item.pop('ura', None)
        item.pop('health', None)
        for k,v in item.items():
            item[k] = np.float64(v)
        item["transmit_time"] = transmit_time[i]
        ephem[i] = item
# pprint(ephem[0])
# pprint(ephem[0].keys())

pprint(code_phase_time)
pprint(transmit_time)

array([0.01116481, 0.00802455, 0.01106949, 0.01065768, 0.00386576,
       0.00414044, 0.01194815])
array([413658.01116481, 413658.00802455, 413658.01106949, 413658.01065768,
       413658.00386576, 413658.00414044, 413658.01194815])


In [2]:
sv_pos = np.zeros((7,3))
sv_pos_new = np.zeros((7,3))
sv_vel = np.zeros((7,3))
sv_clk = np.zeros((7,3))

# get satellite positions based on transmit time
for i in range(7):
    sv_clk[i,:], sv_pos[i,:], sv_vel[i,:], _ = GetNavStates(**ephem[i])
    # sv_pos_new[i,:] = CorrectEarthRotation(-(receive_time - transmit_time[i]), sv_pos[i,:])
    tgd[i] = ephem[i]['tgd']
# pprint(sv_pos - sv_pos_new)

psr = (receive_time - transmit_time + sv_clk[:,0] - tgd)

# get corrected positions with new transmit time
new_transmit_time = receive_time - psr
for i in range(7):
    ephem[i]['transmit_time'] = new_transmit_time[i]
    sv_clk[i,:], sv_pos_new[i,:], sv_vel[i,:], _ = GetNavStates(**ephem[i])
    sv_pos_new[i,:] = eci2ecef(+psr[i], sv_pos_new[i,:])

pprint(transmit_time - new_transmit_time)
pprint(sv_pos - sv_pos_new)

array([ 2.02172319e-04,  2.21833528e-04, -2.18808418e-06,  7.05800310e-04,
        2.90926604e-04,  1.54292618e-04, -5.26625721e-04])
array([[ 8.42214553e+01,  6.82544312e+01,  4.71833296e-01],
       [ 1.34004304e+02,  3.54248914e+01, -6.98642604e-01],
       [ 7.93208713e+01, -8.64582767e+00,  8.90541822e-04],
       [ 1.10462818e+02, -5.06792280e+01,  1.84901835e+00],
       [ 1.13944730e+02, -9.67869103e+01,  9.20217028e-01],
       [ 4.14597710e+01,  8.54644601e+01,  1.66066844e-01],
       [ 1.24031219e+02, -9.07792871e+00,  1.51388185e+00]])


In [3]:
R = np.diag(BETA**2 * DllVariance(CNo, 0.02))
x, _, _ = LeastSquaresPos(sv_pos_new, psr * LIGHT_SPEED, R, np.zeros(4))
true_x = np.asarray([422596.629, -5362864.287, 3415493.797])
true_range = np.linalg.norm(true_x[...] - sv_pos_new, axis=1)

print(f"lla_est = {np.array2string(ecef2lla(x[:3]), precision=8)}")
print(f"x_est   = {np.array2string(x, precision=3)}")
print(f"x_true  = {np.array2string(true_x, precision=3)}")
print(f"diff    = {np.array2string(true_x - x[:3], precision=3)}, norm = {np.array2string(np.linalg.norm(true_x - x[:3]), precision=6)}")
print(f"psr     = {np.array2string(psr*LIGHT_SPEED, precision=0)}")
print(f"r_true  = {np.array2string(true_range, precision=0)}")
print(f"diff    = {np.array2string(true_range - psr*LIGHT_SPEED, precision=3)}")

lla_est = [ 32.58629138 -85.49444246 201.92130991]
x_est   = [  422590.433 -5362870.009  3415498.793    68327.03 ]
x_true  = [  422596.629 -5362864.287  3415493.797]
diff    = [ 6.196  5.722 -4.996], norm = 9.802197
psr     = [20921769. 21869092. 20889080. 21224788. 23136577. 23013270. 20468442.]
r_true  = [20853444. 21800766. 20820763. 21156469. 23068259. 22944942. 20400125.]
diff    = [-68324.543 -68325.953 -68316.492 -68318.863 -68318.184 -68328.404
 -68317.679]
